# B04.1 Multiple sample Clustring - Batch Intergration 多样本聚类 - 去批次效应后聚类

- Script Function: Perform clustering on the multi-sample data after batch integration using the preprocessed h5ad files.    
脚本功能：对经过预处理的h5ad文件进行多样本数据进行批次整合后聚类。  
- Input: Preprocessed adjusted.cellbin.h5ad files.  
输入：预处理后的adjusted.cellbin.h5ad文件  
- Processing: After reading the h5ad files, perform PCA and then conduct batch integration based on the PCA results to remove batch effects. Subsequently, perform clustering and compare the results with those obtained without batch integration.  
处理过程：读取h5ad后，进行PCA后，基于PCA结果进行去批次的批次整合后，进行聚类，并与未进行批次聚类的结果进行对比。  
- Output: Multi-sample h5ad after batch integration and clustering.  
输出：批次整合后，聚类后的多样本h5ad  

## 0. Package importing 环境导入

In [1]:
import stereo as st
import scanpy as sc
import warnings
from skimage import io, transform
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt

from stereo.core.ms_data import MSData
from stereo.core.ms_pipeline import slice_generator
import pandas as pd
import numpy as np

## 1. Data importing 数据导入

Read according to the h5ad file path in dataLink.  
根据dataLink的信息中的h5ad进行读取

In [2]:
dataList = []
dataLink = {
    'CT1':'./processdata/B02.CT1.adjusted.cellbin.h5ad',
    'CT2':'./processdata/B02.CT2.adjusted.cellbin.h5ad',
    'CT3':'./processdata/B02.CT3.adjusted.cellbin.h5ad',
    'EP1':'./processdata/B02.EP1.adjusted.cellbin.h5ad',
    'EP2':'./processdata/B02.EP2.adjusted.cellbin.h5ad',
    'EP3':'./processdata/B02.EP3.adjusted.cellbin.h5ad',
}

In [3]:
# Function to print a separator line
def print_separator():
    print("=" * 80)

# Function to print a header
def print_header(message):
    print_separator()
    print(f"{message:^80}")  # Center the message
    print_separator()

for sample_name, file_path in dataLink.items():
    # Print header for each sample
    print_header(f"Processing Sample: {sample_name}")
    
    # Start reading data
    print(f"++ Starting to read data of sample {sample_name}")
    indata = st.io.read_h5ad(file_path=file_path)
    
    # Perform raw data checkpoint
    indata.tl.raw_checkpoint()
    
    # Assign the sample name to the data object
    indata.samplename = sample_name
    
    # Plot and display the spatial scatter plot
    #print(f"++++ Spatial scatter plot of sample {sample_name}")
    #indata.plt.spatial_scatter()
    #plt.show()  # Force display of the plot
    
    # Print information about the sample data
    print(f"++++ Information of sample {sample_name}")
    print(indata)
    
    # Append the processed data to the dataList
    dataList.append(indata)
    
    # Print footer
    print_separator()

                             Processing Sample: CT1                             
++ Starting to read data of sample CT1
++++ Information of sample CT1
AnnBasedStereoExpData object with n_cells X n_genes = 145065 X 28871
adata: id(139706316836528)
bin_type: cell_bins
offset_x = None
offset_y = None
cells: ['cell_name', 'dnbCount', 'area', 'total_counts', 'n_genes_by_counts', 'pct_counts_mt', 'orig.ident', 'x', 'y']
genes: ['gene_name', 'n_cells', 'n_counts', 'mean_umi']
cells_matrix = ['cell_border', 'spatial']
Layers with keys: 
tl.result: []
                             Processing Sample: CT2                             
++ Starting to read data of sample CT2
++++ Information of sample CT2
AnnBasedStereoExpData object with n_cells X n_genes = 142030 X 28807
adata: id(139708193787808)
bin_type: cell_bins
offset_x = None
offset_y = None
cells: ['cell_name', 'dnbCount', 'area', 'total_counts', 'n_genes_by_counts', 'pct_counts_mt', 'orig.ident', 'x', 'y']
genes: ['gene_name', 'n_cells', '

## 2. Batch Effect Removal 去批次效应

### 2.1 Generation multiple-sample-analysis target 生成多样本处理对象

In [4]:
data = MSData(_data_list=dataList)
data.shape

{'0': (145065, 28871),
 '1': (142030, 28807),
 '2': (104239, 28846),
 '3': (165910, 28819),
 '4': (147190, 28750),
 '5': (112877, 28920)}

### 2.2 Normalization 归一化

In [ ]:
data.tl.normalize_total(scope=slice_generator[:], mode='integrate')
data.tl.log1p(scope=slice_generator[:], mode='integrate')
data.tl.scale(scope=slice_generator[:], mode='integrate')

[2025-01-02 17:58:28][Stereo][166346][MainThread][139709886900032][ms_pipeline][123][INFO]: data_obj(idx=0) in ms_data start to run normalize_total
[2025-01-02 17:58:28][Stereo][166346][MainThread][139709886900032][st_pipeline][41][INFO]: start to run normalize_total...
[2025-01-02 17:59:02][Stereo][166346][MainThread][139709886900032][st_pipeline][44][INFO]: normalize_total end, consume time 34.1716s.
[2025-01-02 17:59:02][Stereo][166346][MainThread][139709886900032][ms_pipeline][123][INFO]: data_obj(idx=0) in ms_data start to run log1p
[2025-01-02 17:59:02][Stereo][166346][MainThread][139709886900032][st_pipeline][41][INFO]: start to run log1p...
[2025-01-02 17:59:09][Stereo][166346][MainThread][139709886900032][st_pipeline][44][INFO]: log1p end, consume time 6.5792s.
[2025-01-02 17:59:09][Stereo][166346][MainThread][139709886900032][ms_pipeline][123][INFO]: data_obj(idx=0) in ms_data start to run scale
[2025-01-02 17:59:09][Stereo][166346][MainThread][139709886900032][st_pipeline][4

### 2.3 PCA 

In [ ]:
data.tl.pca(scope=slice_generator[:], mode='integrate', use_highly_genes=False, n_pcs=50, res_key='pca')

### 2.4 Batches integration 批次整合

Integrate data from different batches together, using pca_res_key to specifically refer to the integrated PCA results. Therefore, it is necessary to store the integrated PCA results in res_key.  Integrated PCA results refer to batch effect removed PCA results with **harmony**.

将不同批次的数据整合在一起，使用pca_res_key特指调整后的PCA结果，因此需要将整合后的PCA结果存储在res_key中。 整合后的PCA结果特指经过**harmony**算法进行批次效应去除的结果。

In [ ]:
data.tl.batches_integrate(scope=slice_generator[:],pca_res_key='pca', res_key='pca_integrated')

### 2.5 UMAP

Run UMAP using the integrated PCA results after batch effect removal.  
在使用批次效应去除后的整合PCA结果进行邻域分析及UMAP运行。  

In [ ]:
data.tl.neighbors(scope=slice_generator[:],pca_res_key='pca_integrated', n_pcs=50, res_key='neighbors_integrated')
data.tl.umap(scope=slice_generator[:],pca_res_key='pca_integrated', neighbors_res_key='neighbors_integrated', res_key='umap_integrated')

In [ ]:
data.plt.batches_umap(scope=slice_generator[:],res_key='umap_integrated')

### 2.6 Clustering 聚类

In [ ]:
data.tl.leiden(scope=slice_generator[:],neighbors_res_key='neighbors_integrated', res_key='leiden_integrated')

In [ ]:
data.plt.cluster_scatter(scope=slice_generator[:], mode='integrate', res_key='leiden_integrated', reorganize_coordinate=3,dot_size=0.1)

### 2.7 Clustering results before batch integration 未进行批次整合前的聚类结果  

Perform neighborhood analysis and run UMAP using the PCA results before batch effect removal. Therefore, when running **data.tl.neighbors**, you need to use **pca_res_key='pca'** instead of **pca_integrated**.
在使用批次效应去除前的PCA结果进行邻域分析及UMAP运行。因此在进行**data.tl.neighbors**运行时需要使用的**pca_res_key='pca'**，而不是**pca_integrated**。

In [ ]:
data.tl.neighbors(pca_res_key='pca', n_pcs=50, res_key='neighbors', n_jobs=-1)
data.tl.umap(pca_res_key='pca', neighbors_res_key='neighbors', res_key='umap')

In [ ]:
data.plt.batches_umap(res_key='umap')

In [ ]:
data.tl.leiden(neighbors_res_key='neighbors', res_key='leiden')

In [ ]:
data.plt.cluster_scatter(scope=slice_generator[:], mode='integrate', res_key='leiden', reorganize_coordinate=3)

In [ ]:
st.io.write_h5ms(
        data,
        output='./processdata/B04.1.SDdata.cluster.h5ad',
        )